# Download Pyspark

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 56.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8736814f84629f4cb63730a71e05848ba22e03eabecd77b4cc67812e7e131458
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


# Imports

In [229]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

import json
import itertools
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np

from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LinearSVC, LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

from google.colab import files

import pyspark.sql.functions as psf
from pyspark.sql.functions import udf, col, lower, regexp_replace, split, monotonically_increasing_id
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec
from nltk.stem.snowball import SnowballStemmer

from pyspark.sql.types import StringType, ArrayType, FloatType, IntegerType

import string

from pyspark.ml.functions import vector_to_array

from pyspark.ml.feature import StringIndexer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [3]:
# Creating spark context
sc = SparkContext()
# Create a spark session
spark = SparkSession(sc)

# Read the data

In [4]:
files.upload()

Saving smsspamcollection.zip to smsspamcollection.zip


{'smsspamcollection.zip': b'PK\x03\x04\x14\x03\x00\x00\x08\x00\x81\xb4o>t\x90k\x96\xd2\x0e\x03\x00\xd3J\x07\x00\x11\x00\x00\x00SMSSpamCollection\xd4]\xc9v\x1bM\x15^\xc3S\x94\x81\x93\x10P:\x92,;\xb69`\xe4!\xb6\xc1C\xf0\x10\'l8-\xa9%u,u\x8b\x1e,\x8b\xc3\xe1\xf0\x0c\xacX\xb0\xe3-8\xac\xe0Mx\x12\xbe\xef\xde[\xd5r\x98\x87\x05\xe4\xff\x13[\xad\xea\xee\x1an\xdd\xe1\xbbCM\xe3\xf9WNrWgU:s\x9f\xeb"\xcf&n\x91\xa7Y\xd5r\xc3"\xfe\xd9*\x8a\\\xff1Ng\xf1`\x96\xb8<\x9b\xad\\\x9a\xb9A=IK\x97\xb9I\x91\xc4\x95[\xe6\xc5l\xe4f\xb1K\xf0\xc5x\x9cT\x11n:L\xb3\xc4U\xd3\xa4H\xdc$\xaf\\<\xcf\xf1\xdb2\xe6w_\xe5;\xaf\x1epG\xc1\x96?\xc8\x1fR\xbct\x99\x8e]\x8d7\xa4lQ.\xd0\xe4]\x91$.\xc9\xaaB\xde\xd9u\xb1[>\xe0\xfd\xc3|\xbepU\xee\x96\xb8\xf8\xae\xef\x0e\xeb\x85\x1b\xa7Y<s\xd5CU\xban\xa7\xac\xdcE\xbcr\xddv{+r\xb7\xc9S\x85Vl\xbf\xf3\xb6\xd3\xed\xf0\x97"\x19&\xe9\xa3\x7f\xf4O\xeb\xa4\xac\xd2<\xfbfY\x8d\\\x85\xd6E\\%\xafn_\x1c\xbe,]\xbcX\xe0\x85\xed\x9d\xdeVw\xa7\xd3n\xbf\xdd\xca\x1f\x93\xa2\xb3\xf3\xb2\x94!\xdc\xb9Q\x9d\

In [5]:
!unzip "smsspamcollection.zip"

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [114]:
df = spark.read.format("csv").option("delimiter", "\t").load("SMSSpamCollection")

# EDA

In [7]:
df.show(10)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
+----+--------------------+
only showing top 10 rows



In [8]:
df.head()

Row(_c0='ham', _c1='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

In [9]:
df.describe()

DataFrame[summary: string, _c0: string, _c1: string]

In [10]:
df.explain()

== Physical Plan ==
FileScan csv [_c0#16,_c1#17] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/SMSSpamCollection], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string>




In [11]:
df.summary()

DataFrame[summary: string, _c0: string, _c1: string]

In [12]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [13]:
# Print the number of rows in the dataframe
df.count()

5574

In [250]:
df.groupBy('_c0').count().orderBy('count').show()

+----+-----+
| _c0|count|
+----+-----+
|spam|  747|
| ham| 4827|
+----+-----+



# Data Preparation

## Data Cleaning

In [261]:
# Drop duplicated rows, if any
df_clean1 = df.dropDuplicates()
df_clean1.count()

5171

In [262]:
# Drop null values, if any
df_clean1 = df_clean1.dropna()
df_clean1.count()

5171

In [263]:
# Rename the columns
df_clean1 = df_clean1.withColumnRenamed(df_clean1.columns[0], 'spam')
df_clean1 = df_clean1.withColumnRenamed(df_clean1.columns[1], 'sms')
df_clean1.show()

+----+--------------------+
|spam|                 sms|
+----+--------------------+
| ham|Dear, will call T...|
| ham|Yeah sure, give m...|
| ham|Good afternoon, m...|
| ham|Geeeee ... I love...|
| ham|S'fine. Anytime. ...|
| ham|Not getting anywh...|
| ham|Thank you so much...|
| ham|Doesn't g have cl...|
| ham|Was doing my test...|
| ham|I have a rather p...|
| ham|Yar lor actually ...|
| ham|Sat right? Okay t...|
| ham|     See you there! |
| ham|  Wat r u doing now?|
| ham|Yes, princess. To...|
| ham|Im on gloucesterr...|
| ham|Actually nvm, got...|
| ham|I just really nee...|
| ham|Party's at my pla...|
| ham|HI BABE UAWAKE?FE...|
+----+--------------------+
only showing top 20 rows



In [264]:
df_clean1.groupBy('spam').count().orderBy('count').show()

+----+-----+
|spam|count|
+----+-----+
|spam|  653|
| ham| 4518|
+----+-----+



In [265]:
# Encode the target
indexer = StringIndexer(inputCol="spam", outputCol="target") 
indexed = indexer.fit(df_clean1).transform(df_clean1) 
df_clean = indexed.withColumn("target", indexed["target"].cast(IntegerType())).select('sms', 'target')
df_clean.show()

+--------------------+------+
|                 sms|target|
+--------------------+------+
|Dear, will call T...|     0|
|Yeah sure, give m...|     0|
|Good afternoon, m...|     0|
|Geeeee ... I love...|     0|
|S'fine. Anytime. ...|     0|
|Not getting anywh...|     0|
|Thank you so much...|     0|
|Doesn't g have cl...|     0|
|Was doing my test...|     0|
|I have a rather p...|     0|
|Yar lor actually ...|     0|
|Sat right? Okay t...|     0|
|     See you there! |     0|
|  Wat r u doing now?|     0|
|Yes, princess. To...|     0|
|Im on gloucesterr...|     0|
|Actually nvm, got...|     0|
|I just really nee...|     0|
|Party's at my pla...|     0|
|HI BABE UAWAKE?FE...|     0|
+--------------------+------+
only showing top 20 rows



In [266]:
df_clean.describe()

DataFrame[summary: string, sms: string, target: string]

## Text preprocessing

In [267]:
# Print sms values 
df_clean.select('sms').collect()

[Row(sms='Dear, will call Tmorrow.pls accomodate.'),
 Row(sms='Yeah sure, give me a couple minutes to track down my wallet'),
 Row(sms='Good afternoon, my love! How goes that day ? I hope maybe you got some leads on a job. I think of you, boytoy and send you a passionate kiss from across the sea'),
 Row(sms='Geeeee ... I love you so much I can barely stand it'),
 Row(sms="S'fine. Anytime. All the best with it."),
 Row(sms='Not getting anywhere with this damn job hunting over here!'),
 Row(sms="Thank you so much. When we skyped wit kz and sura, we didnt get the pleasure of your company. Hope you are good. We've given you ultimatum oh! We are countin down to aburo. Enjoy! This is the message i sent days ago"),
 Row(sms="Doesn't g have class early tomorrow and thus shouldn't be trying to smoke at  &lt;#&gt;"),
 Row(sms='Was doing my test earlier. I appreciate you. Will call you tomorrow.'),
 Row(sms='I have a rather prominent bite mark on my right cheek'),
 Row(sms='Yar lor actually we qu

### Tokenization

In [268]:
tokenizer = Tokenizer(inputCol='sms', outputCol='sms_tokens')
df_words_token = tokenizer.transform(df_clean)
df_words_token.show()

+--------------------+------+--------------------+
|                 sms|target|          sms_tokens|
+--------------------+------+--------------------+
|Dear, will call T...|     0|[dear,, will, cal...|
|Yeah sure, give m...|     0|[yeah, sure,, giv...|
|Good afternoon, m...|     0|[good, afternoon,...|
|Geeeee ... I love...|     0|[geeeee, ..., i, ...|
|S'fine. Anytime. ...|     0|[s'fine., anytime...|
|Not getting anywh...|     0|[not, getting, an...|
|Thank you so much...|     0|[thank, you, so, ...|
|Doesn't g have cl...|     0|[doesn't, g, have...|
|Was doing my test...|     0|[was, doing, my, ...|
|I have a rather p...|     0|[i, have, a, rath...|
|Yar lor actually ...|     0|[yar, lor, actual...|
|Sat right? Okay t...|     0|[sat, right?, oka...|
|     See you there! |     0|  [see, you, there!]|
|  Wat r u doing now?|     0|[wat, r, u, doing...|
|Yes, princess. To...|     0|[yes,, princess.,...|
|Im on gloucesterr...|     0|[im, on, gloucest...|
|Actually nvm, got...|     0|[a

### Remove punctuation


In [269]:
punc_udf = udf(lambda tokens: [token.strip(string.punctuation) for token in tokens], ArrayType(StringType()))

df_no_punc = df_words_token.withColumn("sms_no_punc", punc_udf("sms_tokens"))

df_no_punc.show()

+--------------------+------+--------------------+--------------------+
|                 sms|target|          sms_tokens|         sms_no_punc|
+--------------------+------+--------------------+--------------------+
|Dear, will call T...|     0|[dear,, will, cal...|[dear, will, call...|
|Yeah sure, give m...|     0|[yeah, sure,, giv...|[yeah, sure, give...|
|Good afternoon, m...|     0|[good, afternoon,...|[good, afternoon,...|
|Geeeee ... I love...|     0|[geeeee, ..., i, ...|[geeeee, , i, lov...|
|S'fine. Anytime. ...|     0|[s'fine., anytime...|[s'fine, anytime,...|
|Not getting anywh...|     0|[not, getting, an...|[not, getting, an...|
|Thank you so much...|     0|[thank, you, so, ...|[thank, you, so, ...|
|Doesn't g have cl...|     0|[doesn't, g, have...|[doesn't, g, have...|
|Was doing my test...|     0|[was, doing, my, ...|[was, doing, my, ...|
|I have a rather p...|     0|[i, have, a, rath...|[i, have, a, rath...|
|Yar lor actually ...|     0|[yar, lor, actual...|[yar, lor, act

### Removing stopwords


Will experiment the model with and without stopwords

In [270]:
remover = StopWordsRemover(inputCol='sms_no_punc', outputCol='sms_no_stopw')
df_words_no_stopw = remover.transform(df_no_punc)
df_words_no_stopw.show()

+--------------------+------+--------------------+--------------------+--------------------+
|                 sms|target|          sms_tokens|         sms_no_punc|        sms_no_stopw|
+--------------------+------+--------------------+--------------------+--------------------+
|Dear, will call T...|     0|[dear,, will, cal...|[dear, will, call...|[dear, call, tmor...|
|Yeah sure, give m...|     0|[yeah, sure,, giv...|[yeah, sure, give...|[yeah, sure, give...|
|Good afternoon, m...|     0|[good, afternoon,...|[good, afternoon,...|[good, afternoon,...|
|Geeeee ... I love...|     0|[geeeee, ..., i, ...|[geeeee, , i, lov...|[geeeee, , love, ...|
|S'fine. Anytime. ...|     0|[s'fine., anytime...|[s'fine, anytime,...|[s'fine, anytime,...|
|Not getting anywh...|     0|[not, getting, an...|[not, getting, an...|[getting, anywher...|
|Thank you so much...|     0|[thank, you, so, ...|[thank, you, so, ...|[thank, much, sky...|
|Doesn't g have cl...|     0|[doesn't, g, have...|[doesn't, g, have...

### Stemming 


In [271]:
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
df_stemmed = df_words_no_stopw.withColumn("sms_stemmed", stemmer_udf("sms_no_stopw"))
df_stemmed.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|                 sms|target|          sms_tokens|         sms_no_punc|        sms_no_stopw|         sms_stemmed|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+
|Dear, will call T...|     0|[dear,, will, cal...|[dear, will, call...|[dear, call, tmor...|[dear, call, tmor...|
|Yeah sure, give m...|     0|[yeah, sure,, giv...|[yeah, sure, give...|[yeah, sure, give...|[yeah, sure, give...|
|Good afternoon, m...|     0|[good, afternoon,...|[good, afternoon,...|[good, afternoon,...|[good, afternoon,...|
|Geeeee ... I love...|     0|[geeeee, ..., i, ...|[geeeee, , i, lov...|[geeeee, , love, ...|[geeeee, , love, ...|
|S'fine. Anytime. ...|     0|[s'fine., anytime...|[s'fine, anytime,...|[s'fine, anytime,...|[s'fine, anytim, ...|
|Not getting anywh...|     0|[not, getting, an...|[not, getting, an...|[getting, anywher

### Feature Extraction using Word2Vec

In [272]:
# Learn a mapping from words to Vectors.
l_emb = 5
word2Vec = Word2Vec(vectorSize=l_emb, minCount=0, inputCol="sms_stemmed", outputCol="sms_embedding")
model = word2Vec.fit(df_stemmed)

df_clean =  model.transform(df_stemmed)
#df_clean = df_clean.withColumn('sms_embedding', vector_to_array('sms_embedding'))
result = df_clean.select("sms_stemmed", "sms_embedding")

# Print the results
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Streaming output truncated to the last 5000 lines.
Vector: [0.11423841620916905,-0.03368672829769228,0.15892574875413076,0.11268772697329278,0.04697431556111121]

Text: [hello, lover, goe, new, job, happi, think, wake, slave, send, teas, kiss, across, sea] => 
Vector: [-0.09191697556525469,-0.12023872110460485,0.0011455051800502197,-0.03411512372882238,0.15749322243833117]

Text: [take, care, financi, problem.i, help] => 
Vector: [0.005296563729643822,-0.10796677228063345,0.09007256189361215,-0.06824403703212738,-0.042871759831905366]

Text: [dint, slept, afternoon] => 
Vector: [0.0044890207548936205,-0.10751860712965329,0.035859911857793726,-0.02302495103018979,0.10804541409015656]

Text: [happen, gotten, job, begun, registr, perman, resid] => 
Vector: [-0.007288348029500671,-0.009564084932208061,0.03416164122921015,0.01095492352864572,0.046416178811341524]

Text: [u, ask, darren, go, n, pick, u, lor, oso, sian, tmr, haf, 2, meet, lect] => 
Vector: [0.12317303183178106,-0.143493904794

In [273]:
df_clean = df_clean.select('sms_embedding', 'target')
df_clean.show()

+--------------------+------+
|       sms_embedding|target|
+--------------------+------+
|[0.10354310087859...|     0|
|[-0.0373078279163...|     0|
|[-0.1820395867236...|     0|
|[-0.2732755985731...|     0|
|[0.02057673782110...|     0|
|[-0.0339295655488...|     0|
|[-0.0464237560253...|     0|
|[-0.1369312564945...|     0|
|[0.09597934260964...|     0|
|[-0.0068581603506...|     0|
|[0.06451604015786...|     0|
|[0.00830243524978...|     0|
|[-0.1732245683670...|     0|
|[0.12624649470672...|     0|
|[-0.0702508582423...|     0|
|[0.01670102030038...|     0|
|[-0.1539178796940...|     0|
|[-0.0529117565602...|     0|
|[-0.0405792580277...|     0|
|[0.01747605424108...|     0|
+--------------------+------+
only showing top 20 rows



In [274]:
df_clean.select('sms_embedding').collect()

[Row(sms_embedding=DenseVector([0.1035, -0.127, 0.1663, 0.1672, 0.0493])),
 Row(sms_embedding=DenseVector([-0.0373, -0.0924, 0.071, -0.0297, 0.062])),
 Row(sms_embedding=DenseVector([-0.182, -0.1424, -0.0598, -0.1173, 0.2303])),
 Row(sms_embedding=DenseVector([-0.2733, -0.108, -0.0704, -0.1247, 0.2996])),
 Row(sms_embedding=DenseVector([0.0206, 0.0043, 0.1065, -0.0073, 0.0138])),
 Row(sms_embedding=DenseVector([-0.0339, -0.0859, 0.0114, 0.0039, 0.0501])),
 Row(sms_embedding=DenseVector([-0.0464, -0.1525, 0.0355, -0.0517, 0.0995])),
 Row(sms_embedding=DenseVector([-0.1369, -0.0711, -0.0554, -0.0986, 0.3185])),
 Row(sms_embedding=DenseVector([0.096, -0.0338, 0.1388, 0.1078, -0.0167])),
 Row(sms_embedding=DenseVector([-0.0069, -0.0688, -0.0071, 0.0247, 0.0298])),
 Row(sms_embedding=DenseVector([0.0645, -0.0965, 0.1571, -0.1427, -0.0429])),
 Row(sms_embedding=DenseVector([0.0083, -0.1024, 0.053, -0.0183, 0.049])),
 Row(sms_embedding=DenseVector([-0.1732, -0.2366, -0.0743, -0.1114, 0.0392])

# Train Test Split

In [297]:
# To visualize predictions 
train_string, test_string = df_clean1.randomSplit([0.8, 0.2], seed=1)

In [298]:
train_string.count(), test_string.count()

(4110, 1061)

In [299]:
# To train and test the model
train, test = df_clean.randomSplit([0.8, 0.2], seed=1)

In [300]:
train.count(), test.count()

(4110, 1061)

In [302]:
test_string.groupBy('spam').count().orderBy('count').show()

+----+-----+
|spam|count|
+----+-----+
|spam|  134|
| ham|  927|
+----+-----+



# Model

## Linear Support Vector Machine

In [305]:
train.describe()

DataFrame[summary: string, target: string]

In [306]:
lsvc = LinearSVC(featuresCol='sms_embedding', labelCol='target',maxIter=10, regParam=0.1)

# Fit the model
lsvcModel = lsvc.fit(train)

# Predict
predictions = lsvcModel.transform(test)

# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

Coefficients: [1.093103318689387,3.2325655002533398,1.5445562381794251,4.588916414471617,0.9412507404390484]
Intercept: -0.9792480510623005


#### Evaluate the model

In [211]:
# Evaluate our model
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', 
    predictionCol='prediction', 
    metricName='accuracy')

In [212]:
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy of Linear SVC model = {}%'.format(accuracy*100))

Test Accuracy of Linear SVC model = 88.25271470878579%


#### Print the predictions

In [308]:
test_string = test_string.withColumn("id1",monotonically_increasing_id())
predictions = predictions.withColumn("id2",monotonically_increasing_id())

In [309]:
df_joined = test_string.join(predictions ,test_string.id1 ==  predictions.id2,"inner").select('sms', 'spam', 'prediction')

In [310]:
df_joined.show(50)

+--------------------+----+----------+
|                 sms|spam|prediction|
+--------------------+----+----------+
| came to look at ...| ham|       0.0|
|"Response" is one...| ham|       0.0|
|"SYMPTOMS" when U...| ham|       0.0|
|"Speak only when ...| ham|       0.0|
|"Wen u miss someo...| ham|       0.0|
|(No promises on w...| ham|       0.0|
|* Am on a train b...| ham|       0.0|
|      * Am on my way| ham|       0.0|
|, ,  and  picking...| ham|       0.0|
|, im .. On the sn...| ham|       0.0|
|1's finish meetin...| ham|       0.0|
|1. Tension face 2...| ham|       0.0|
|2 and half years ...| ham|       0.0|
|4 oclock at mine....| ham|       0.0|
|                 :) | ham|       0.0|
| :-( sad puppy noise| ham|       0.0|
|A bloo bloo bloo ...| ham|       0.0|
|A boy was late 2 ...| ham|       0.0|
|A gram usually ru...| ham|       0.0|
|A pure hearted pe...| ham|       0.0|
|Actually, my mobi...| ham|       0.0|
|Aight ill get on ...| ham|       0.0|
|Aight should I ju...| ha

In [312]:
df_joined.groupBy('prediction').count().orderBy('count').show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|   29|
|       0.0| 1032|
+----------+-----+



## Logistic Regression

In [313]:
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(featuresCol='sms_embedding', labelCol='target', maxIter=10, regParam=0.01)

# Fit the model
model1 = lr.fit(train)

# Predict
predictions2 = model1.transform(test)

# for row in result:
#     print("features=%s, label=%s -> prob=%s, prediction=%s"
#           % (row.features, row.label, row.myProbability, row.prediction))

#### Evaluate the model


In [314]:
# Evaluate the model
accuracy2 = evaluator.evaluate(predictions2)
print('Test Accuracy of Logistic Regression model = {}%'.format(accuracy2*100))

Test Accuracy of Logistic Regression model = 91.98868991517436%


#### Print the predictions

In [319]:
test_string = test_string.withColumn("id1",monotonically_increasing_id())
predictions2 = predictions2.withColumn("id2",monotonically_increasing_id())

In [320]:
df_joined = test_string.join(predictions2 ,test_string.id1 ==  predictions2.id2,"inner").select('sms', 'spam', 'prediction')

In [321]:
df_joined.show(50)

+--------------------+----+----------+
|                 sms|spam|prediction|
+--------------------+----+----------+
| came to look at ...| ham|       0.0|
|"Response" is one...| ham|       0.0|
|"SYMPTOMS" when U...| ham|       0.0|
|"Speak only when ...| ham|       0.0|
|"Wen u miss someo...| ham|       0.0|
|(No promises on w...| ham|       0.0|
|* Am on a train b...| ham|       0.0|
|      * Am on my way| ham|       0.0|
|, ,  and  picking...| ham|       0.0|
|, im .. On the sn...| ham|       0.0|
|1's finish meetin...| ham|       0.0|
|1. Tension face 2...| ham|       0.0|
|2 and half years ...| ham|       0.0|
|4 oclock at mine....| ham|       0.0|
|                 :) | ham|       0.0|
| :-( sad puppy noise| ham|       0.0|
|A bloo bloo bloo ...| ham|       0.0|
|A boy was late 2 ...| ham|       0.0|
|A gram usually ru...| ham|       0.0|
|A pure hearted pe...| ham|       0.0|
|Actually, my mobi...| ham|       0.0|
|Aight ill get on ...| ham|       0.0|
|Aight should I ju...| ha

In [322]:
df_joined.groupBy('prediction').count().orderBy('count').show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|   82|
|       0.0|  979|
+----------+-----+



## Conclusion

* Logistic Regression model gave better accuracy